In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [7]:
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.client import TradingClient 
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass
#trading_client
from alpaca.trading.client import TradingClient
#LimitOrderRequest
from alpaca.trading.requests import LimitOrderRequest

from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest


In [8]:
import alpaca_trade_api as tradeapi

In [9]:
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=64):
        super(QNetwork, self).__init__()
        self.linear1 = nn.Linear(state_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, action_dim)
        
    def forward(self, state):
        x = torch.relu(self.linear1(state))
        x = torch.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [10]:
class DQNAgent():
    def __init__(self, state_dim, action_dim, learning_rate=0.001, gamma=0.99, epsilon=1.0, epsilon_decay=0.9999, epsilon_min=0.01):
        self.q_network = QNetwork(state_dim, action_dim)
        self.target_network = QNetwork(state_dim, action_dim)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.action_dim = action_dim
        
    def act(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.action_dim)
        state = torch.FloatTensor(state)
        q_values = self.q_network(state)
        return torch.argmax(q_values).item()
    
    def train(self, replay_buffer, batch_size):
        state, action, next_state, reward, done = replay_buffer.sample(batch_size)
        
        state = torch.FloatTensor(state)
        next_state = torch.FloatTensor(next_state)
        action = torch.LongTensor(action)
        reward = torch.FloatTensor(reward)
        done = torch.FloatTensor(done)
        
        q_values = self.q_network(state)
        next_q_values = self.target_network(next_state).detach()
        target_q_values = reward + (1 - done) * self.gamma * torch.max(next_q_values, dim=1)[0]
        target_q_values = target_q_values.unsqueeze(-1)
        q_value = torch.gather(q_values, dim=1, index=action.unsqueeze(-1))
        
        loss = nn.MSELoss()(q_value, target_q_values)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        self.update_target_network()
        self.update_epsilon()
        
    def update_target_network(self):
        tau = 0.001
        for target_param, param in zip(self.target_network.parameters(), self.q_network.parameters()):
            target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
    
    def update_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


In [11]:
import random


class ReplayBuffer():
    def __init__(self, buffer_size=100000):
        self.buffer_size = buffer_size
        self.buffer = []
        self.position = 0
        
    def add(self, state, action, next_state, reward, done):
        experience = (state, action, next_state, reward, done)
        if len(self.buffer) < self.buffer_size:
            self.buffer.append(None)
        self.buffer[self.position] = experience
        self.position = (self.position + 1) % self.buffer_size
        
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, next_state, reward, done = zip(*batch)
        return np.array(state), np.array(action), np.array(next_state), np.array(reward), np.array(done)



In [12]:
API_KEY = "PKUM529GXGRDHTQHPPI8"
SECRET_KEY = "hUV2Q6TgSVOrdl9D7r0nNoPMYlgN6Mr1HpOnX0Pl"
base_url = 'https://paper-api.alpaca.markets' 

trading_client = TradingClient(API_KEY, SECRET_KEY, paper=True)
historical_data_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)
api = tradeapi.REST(API_KEY, SECRET_KEY, base_url, api_version='v2')

In [13]:
# Getting account information and printing it
account = trading_client.get_account()
for property_name, value in account:
  print(f"\"{property_name}\": {value}")

"id": 6bf64691-3bb4-41e4-b234-6af3076f4ec8
"account_number": PA3ZPLZ3XCLN
"status": ACTIVE
"crypto_status": ACTIVE
"currency": USD
"buying_power": 200000
"regt_buying_power": 200000
"daytrading_buying_power": 0
"non_marginable_buying_power": 100000
"cash": 100000
"accrued_fees": 0
"pending_transfer_out": None
"pending_transfer_in": 0
"portfolio_value": 100000
"pattern_day_trader": False
"trading_blocked": False
"transfers_blocked": False
"account_blocked": False
"created_at": 2023-03-15 01:54:56.539659+00:00
"trade_suspended_by_user": False
"multiplier": 2
"shorting_enabled": True
"equity": 100000
"last_equity": 100000
"long_market_value": 0
"short_market_value": 0
"initial_margin": 0
"maintenance_margin": 0
"last_maintenance_margin": 0
"sma": 0
"daytrade_count": 0


In [14]:
api = trading_client

In [15]:
def get_crypto_data(symbols:list = ['BTC/USD'], sleep_time:int = 3):
    time.sleep(sleep_time)
    crypto_data = api.get_latest_crypto_trades(symbols=symbols)

    print(crypto_data)

    return crypto_data

get_crypto_data()

In [16]:
def sell(symbol, api):
        # preparing orders
    market_order_data = MarketOrderRequest(
                        symbol="SPY",
                        qty=1,
                        side=OrderSide.SELL,
                        time_in_force=TimeInForce.GTC
                        )

    # Market order
    market_order = trading_client.submit_order(
                    order_data=market_order_data
                )

    


In [17]:
def buy(symbol):
        # preparing market order
    market_order_data = MarketOrderRequest(
                        symbol="SPY",
                        qty=0.023,
                        side=OrderSide.BUY,
                        time_in_force=TimeInForce.DAY
                        )

    # Market order
    market_order = trading_client.submit_order(
                    order_data=market_order_data
                )

    # preparing limit order
    limit_order_data = LimitOrderRequest(
                        symbol="BTC/USD",
                        limit_price=17000,
                        notional=4000,
                        side=OrderSide.SELL,
                        time_in_force=TimeInForce.FOK
                    )

    # Limit order
    limit_order = trading_client.submit_order(
                    order_data=limit_order_data
                )

In [18]:
def get_stock_data(symbol):
    data = historical_data_client.get_stock_latest_quote(StockLatestQuoteRequest(symbol_or_symbols=symbol))
    price = data[symbol].ask_price
    return data,'-----', price

get_stock_data("AAL")
    

({'AAL': {   'ask_exchange': 'V',
      'ask_price': 0.0,
      'ask_size': 0.0,
      'bid_exchange': 'V',
      'bid_price': 0.0,
      'bid_size': 0.0,
      'conditions': ['R'],
      'symbol': 'AAL',
      'tape': 'C',
      'timestamp': datetime.datetime(2023, 3, 15, 20, 0, 0, 8973, tzinfo=datetime.timezone.utc)}},
 '-----',
 0.0)

In [19]:
from alpaca.data.live import StockDataStream

wss_client = StockDataStream(API_KEY, SECRET_KEY)

async def quote_data_handler(data):
    
    print(data)

wss_client.subscribe_quotes(quote_data_handler, "SPY")

wss_client.run()


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
import time


class Bot():

    def __init__(self, api, symbol, window_size, batch_size, epsilon, epsilon_decay, epsilon_min, learning_rate, gamma, memory_size):
        self.api = api
        self.symbol = symbol
        self.window_size = window_size
        self.stock_price_memory = []
        self.batch_size = batch_size
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.memory_size = memory_size
        self.memory = ReplayBuffer(self.memory_size)
        self.model = DQNAgent(self.window_size, 3, self.learning_rate, self.gamma, self.epsilon, self.epsilon_decay, self.epsilon_min)

    def act(self, state):
        return self.model.act(state)
    
    def train(self):
        if len(self.memory.buffer) < self.batch_size:
            return
        self.model.train(self.memory, self.batch_size)

    def remember(self, state, action, reward, next_state, done):
        self.memory.add(state, action, next_state, reward, done)

    def get_state(self, data, t, n):
        d = t - n + 1
        block = data[d:t+1] if d >= 0 else -d * [data[0]] + data[0:t+1]
        res = []
        for i in range(n - 1):
            res.append(block[i+1] - block[i])
        return np.array([res])
    
    def run(self):
        while self.stock_price_memory < self.window_size:
            self.stock_price_memory.append(get_stock_data(self.symbol))
            time.sleep(30)
        self.stock_price_memory = np.array(self.stock_price_memory)
        
        data = get_stock_data(self.symbol)
        l = len(data) - 1
        batch_size = 32
        state = self.get_state(data, 0, self.window_size + 1)
        total_profit = 0
        self.model.q_network.train()
        for t in range(l):
            action = self.act(state)
            next_state = self.get_state(data, t+1, self.window_size + 1)
            reward = 0
            if action == 1:
                buy(self.symbol)
                print("Buy")
            elif action == 2:
                sell(self.symbol, self.api)
                print("Sell")
            else:
                print("Hold")
            done = True if t == l - 1 else False
            self.remember(state, action, reward, next_state, done)
            state = next_state
            self.train()
            if done:
                print("--------------------------------")
                print("Total Profit: {}".format(total_profit))
                print("--------------------------------")
        self.model.q_network.eval()
        return total_profit
    
    def load(self, name):
        self.model.q_network.load_state_dict(torch.load(name))

    def save(self, name):
        torch.save(self.model.q_network.state_dict(), name)

bot = Bot(api, "SPY", 10, 32, 1, 0.995, 0.01, 0.001, 0.9, 100000)
bot.run()

    

    

TypeError: '<' not supported between instances of 'list' and 'int'